In [1]:
import ee
import geemap
import math
ee.Initialize()

### Imports

In [2]:
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
admin = ee.FeatureCollection("FAO/GAUL/2015/level0")
active_mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

In [3]:
roi1 = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

roi2 = ee.Geometry.Polygon(
        [[[30.246670960050185, 1.7911944738716732],
          [30.246670960050185, 1.7103797163160706],
          [30.356362579923232, 1.7103797163160706],
          [30.356362579923232, 1.7911944738716732]]])

all = ee.Geometry.Polygon(
        [[[22.92587933089792, 5.5911393992628495],
          [22.92587933089792, -13.776973382582892],
          [31.45126995589792, -13.776973382582892],
          [31.45126995589792, 5.5911393992628495]]])

In [4]:
rgb_vis_ls5 = {
    'min': 8211.7,
    'max': 11077.3,
    'bands': ['SR_B3', 'SR_B2', 'SR_B1']
}

rgb_vis_s2 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

vvVis = {
    'opacity': 1,
    'bands': ["VV"],
    'min': -13.375350094403828,
    'max': -3.6299557970059952,
    'gamma': 1
}

vhVis = {
    'opacity': 1,
    'bands': ["VH"],
    'min': -21.945806268850372,
    'max': -10.8330283399149,
    'gamma': 1
}


In [5]:
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.566649352977876, 3.131936594576548]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.594069479589553, 3.143136886698536]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.578229613742543, 3.109209943820683]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.581158585986806, 3.1067030888196494]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.55884031463487, 3.11517956941471]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.581819366025293, 3.132088091449441]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.595938514279688, 3.131445322675145]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.59962923388418, 3.1347448648629315]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.565039466428125, 3.104877202480009]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.607741346488673, 3.1193612914096605]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

In [6]:
bare_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.588761955782978, 3.110552597531067]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.590113789126484, 3.1101669284095776]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.570400011069342, 3.114240974803364]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.56574369621949, 3.114048140935224]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.566300979571633, 3.1296952794920045]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.61279975505381, 3.134623171579494]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.620696178393654, 3.1514206812868597]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.60730659099131, 3.15656272195781]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.609366527514748, 3.1041127202608414]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.561301341967873, 3.0982847807562592]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

### Classification/Areal Change

In [7]:
training = bare.merge(vegetation)

composite1985 = ls5 \
        .filter(ee.Filter.bounds(roi1)) \
        .filter(ee.Filter.date('1985-01-01', '1986-01-01')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(roi1)

# Overlay the point on the image to get training data.
training = composite1985.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_ls = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite1985.bandNames()
})

# Classify the image.
classified = composite1985.classify(classifier_ls)

# Map.addLayer(classified,
#   {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, '1985 LS5 Classified')

# Map

In [8]:
training = bare_s.merge(vegetation_s)

composite2021 = s2 \
        .filter(ee.Filter.bounds(roi1)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(roi1) \

# Overlay the point on the image to get training data.
training = composite2021.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_s2 = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite2021.bandNames()
})

# Classify the image.
classified = composite2021.classify(classifier_s2)

# Map.addLayer(classified,
#   {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, '2021 S2 Classified')

# Map

In [9]:
def calculate_percentage_change(feature):
    g = feature.geometry()

    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(g)
    
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g) \
    
    left_classified = composite_ls.classify(classifier_ls)
    right_classified = composite_s2.classify(classifier_s2)
    
    # Reclassify from 0-1 to 1-2
    left_classes = left_classified.remap([0, 1], [1, 2])
    right_classes = right_classified.remap([0, 1], [1, 2])

    initial_vegetation = left_classes.eq(2)
    bare = right_classes.eq(1)
    vegetation_to_bare = initial_vegetation.And(bare)
    
    area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())

    area = area_image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

    area_initial_vegetation = left_classified.eq(1).multiply(ee.Image.pixelArea())

    area2 = area_initial_vegetation.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)
    
    percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)
    
    return feature.set('percent loss',  percent_loss)

In [10]:
def calculate_ndmi_change(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(g)
    ndmi_ls = composite_ls.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDMI')
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g)
    ndmi_s2 = composite_s2.normalizedDifference(['B8', 'B11']).rename('NDMI')

    # Average NDMI
    stats_ls = ndmi_ls.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi_ls.geometry(),
        'scale': 30
    })    
    stats_s2 = ndmi_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi_s2.geometry(),
        'scale': 30
    })
    
    avg_ls = ee.Number(stats_ls.get('NDMI'))
    avg_s2 = ee.Number(stats_s2.get('NDMI'))
    
    # NDMI Loss
    ndmi_loss = avg_ls.subtract(avg_s2)
    
    return feature.set('ndmi loss',  ndmi_loss)

In [11]:
def calculate_nir_swir2_change(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(g)
    nir_swir2_ls = composite_ls.normalizedDifference(['SR_B4', 'SR_B7']).rename('NIR/SWIR2')
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g)
    nir_swir2_s2 = composite_s2.normalizedDifference(['B8', 'B12']).rename('NIR/SWIR2')

    # Average NIR/SWIR2
    stats_ls = nir_swir2_ls.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': nir_swir2_ls.geometry(),
        'scale': 30
    })    
    stats_s2 = nir_swir2_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': nir_swir2_s2.geometry(),
        'scale': 30
    })
    
    avg_ls = ee.Number(stats_ls.get('NIR/SWIR2'))
    avg_s2 = ee.Number(stats_s2.get('NIR/SWIR2'))
    
    # NIR/SWIR2 Loss
    nir_swir2_loss = avg_ls.subtract(avg_s2)
    
    return feature.set('nir/swir2 loss',  nir_swir2_loss)

In [12]:
def calculate_swir1_b(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g)
    swir1_b_s2 = composite_s2.normalizedDifference(['B11', 'B2']).rename('SWIR1/B')

    # Average SWIR1/B   
    stats_s2 = swir1_b_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': swir1_b_s2.geometry(),
        'scale': 30
    })

    avg_s2 = ee.Number(stats_s2.get('SWIR1/B'))
    
    # SWIR1/B Loss
    swir1_b = avg_s2
    
    return feature.set('swir1/b',  swir1_b)

In [13]:
def calculate_nir_g(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g)
    nir_g_s2 = composite_s2.normalizedDifference(['B8', 'B3']).rename('NIR/G')

    # Average NIR/G   
    stats_s2 = nir_g_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': nir_g_s2.geometry(),
        'scale': 30
    })
    
    avg_s2 = ee.Number(stats_s2.get('NIR/G'))
    
    return feature.set('nir/g',  avg_s2)

In [36]:
def calculate_sar_vh(feature):
    g = feature.geometry()
    
    # Images and Bands
    filtered = s1 \
        .filter(ee.Filter.eq('instrumentMode','IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filter(ee.Filter.eq('resolution_meters',10)) \
        .filter(ee.Filter.intersects('.geo', g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .select('VH')

    composite = filtered.median().clip(g)
    
    #area of image
    total_area = g.area()
    total_SqKm = ee.Number(total_area).divide(1e6)
    
    #area of possible mines
    area_mines = composite.lt(-19).rename('mines')
    connect = area_mines.connectedPixelCount(25);
    area_mines = area_mines.updateMask(connect.gt(8));
    area_mines = area_mines.multiply(ee.Image.pixelArea())

    area = area_mines.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 30,
      'maxPixels': 1e10
    })

    mines_SqKm = ee.Number(area.get('mines')).divide(1e6)

    percent_mine = ee.Number(mines_SqKm.divide(total_SqKm).multiply(100))
    
    return feature.set('vh_percent', percent_mine)

### Squares

In [15]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares

edit: remove some stuff from geometry produced
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]])
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

### Revised Workflow

##### vegetation loss, NDMI loss, NIR/SWIR2 (normalized) loss, SWIR1/BLUE (normalized) high in 2021, NIR/G (normalized) loss

In [16]:
def filter_by_vegetation_loss(squares, threshold):
    with_percent_change = squares.map(calculate_percentage_change)
    passed = with_percent_change.filter(ee.Filter.gt('percent loss', threshold))
    return passed

In [17]:
def filter_by_ndmi_loss(squares, threshold):
    with_change = squares.map(calculate_ndmi_change)
    passed = with_change.filter(ee.Filter.gt('ndmi loss', threshold))
    return passed

In [18]:
def filter_by_nir_swir2_loss(squares, threshold):
    with_change = squares.map(calculate_nir_swir2_change)
    passed = with_change.filter(ee.Filter.gt('nir/swir2 loss', threshold))
    return passed

In [19]:
def filter_by_swir1_b(squares, threshold):
    with_change = squares.map(calculate_swir1_b)
    passed = with_change.filter(ee.Filter.lt('swir1/b', threshold))
    return passed

In [20]:
def filter_by_nir_g(squares, threshold):
    with_change = squares.map(calculate_nir_g)
    passed = with_change.filter(ee.Filter.lte('nir/g', threshold))
    return passed

In [37]:
def filter_by_vh_percent(squares, threshold):
    with_change = squares.map(calculate_sar_vh)
    passed = with_change.filter(ee.Filter.gt('vh_percent', threshold))
    return passed

In [22]:
def create_median_composite(feature):
    filtered = s2 \
        .filter(ee.Filter.bounds(feature.geometry())) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*')
    
    composite = filtered.median().clip(feature.geometry())
    return composite

In [23]:
def create_whole_composite(geometry):
    composite = s2 \
        .filter(ee.Filter.bounds(geometry)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median().clip(geometry)
    return composite

In [24]:
def create_sar_composite(geometry):
    filtered = s1 \
        .filter(ee.Filter.eq('instrumentMode','IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filter(ee.Filter.eq('resolution_meters',10)) \
        .filter(ee.Filter.intersects('.geo', geometry)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .select(['VV', 'VH'])

    composite = filtered.median().clip(geometry)
    
    return composite

In [27]:
Map = geemap.Map()
Map.centerObject(roi1, 13)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(roi1, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
passed_ndmi_loss = filter_by_ndmi_loss(passed_sar_vh, 0.05)
passed_nir_swir2_loss = filter_by_nir_swir2_loss(passed_ndmi_loss, 0)
passed_swir1_b = filter_by_swir1_b(passed_nir_swir2_loss, 0.65)
passed_nir_g = filter_by_nir_g(passed_swir1_b, 0.45)

#composites = ee.ImageCollection(segments.map(create_median_composite))
composites = create_whole_composite(roi1)

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', False, opacity=0.7)
# Map.addLayer(passed_nir_swir2_loss, {'color': 'yellow'}, 'Passed NIR/SWIR2 Loss', False, opacity=0.7)
# Map.addLayer(passed_swir1_b, {'color': 'green'}, 'Passed SWIR1/B', False, opacity=0.7)
Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…

In [28]:
Map = geemap.Map()
Map.centerObject(roi2, 13)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

# was .1
segments = ee.FeatureCollection(create_segments(roi2, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
passed_ndmi_loss = filter_by_ndmi_loss(passed_sar_vh, 0.05)
passed_nir_swir2_loss = filter_by_nir_swir2_loss(passed_ndmi_loss, 0)
passed_swir1_b = filter_by_swir1_b(passed_nir_swir2_loss, 0.65)
passed_nir_g = filter_by_nir_g(passed_swir1_b, 0.45)

#composites = ee.ImageCollection(segments.map(create_median_composite))
composites = create_whole_composite(roi2)

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', False, opacity=0.7)
# Map.addLayer(passed_nir_swir2_loss, {'color': 'yellow'}, 'Passed NIR/SWIR2 Loss', False, opacity=0.7)
# Map.addLayer(passed_swir1_b, {'color': 'green'}, 'Passed SWIR1/B', False, opacity=0.7)
Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

KeyboardInterrupt: 

In [30]:
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])

Map = geemap.Map()
Map.centerObject(rishiArea, 13)

segments = ee.FeatureCollection(create_segments(rishiArea, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
passed_ndmi_loss = filter_by_ndmi_loss(passed_sar_vh, 0.05)
passed_nir_swir2_loss = filter_by_nir_swir2_loss(passed_ndmi_loss, 0)
passed_swir1_b = filter_by_swir1_b(passed_nir_swir2_loss, 0.65)
passed_nir_g = filter_by_nir_g(passed_swir1_b, 0.45)

#composites = ee.ImageCollection(segments.map(create_median_composite))
composites = create_whole_composite(rishiArea)

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', False, opacity=0.7)
# Map.addLayer(passed_nir_swir2_loss, {'color': 'yellow'}, 'Passed NIR/SWIR2 Loss', False, opacity=0.7)
# Map.addLayer(passed_swir1_b, {'color': 'green'}, 'Passed SWIR1/B', False, opacity=0.7)
Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[-7.55000091393896, 27.399999999997924], controls=(WidgetControl(options=['position', 'transparent_…

In [31]:
emily_area = ee.Geometry.Polygon(
        [[[25.599767416235804, -10.585427828449394],
          [25.599767416235804, -10.82359555219894],
          [25.977297323462366, -10.82359555219894],
          [25.977297323462366, -10.585427828449394]]])

Map = geemap.Map()
Map.centerObject(emily_area, 11)

s = create_segments(emily_area, 0.5)

segments = ee.FeatureCollection(s).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
passed_ndmi_loss = filter_by_ndmi_loss(passed_sar_vh, 0.05)
passed_nir_swir2_loss = filter_by_nir_swir2_loss(passed_ndmi_loss, 0)
passed_swir1_b = filter_by_swir1_b(passed_nir_swir2_loss, 0.65)
passed_nir_g = filter_by_nir_g(passed_swir1_b, 0.45)

#composites = ee.ImageCollection(segments.map(create_median_composite))
composites = create_whole_composite(emily_area)

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', False, opacity=0.7)
# Map.addLayer(passed_nir_swir2_loss, {'color': 'yellow'}, 'Passed NIR/SWIR2 Loss', False, opacity=0.7)
# Map.addLayer(passed_swir1_b, {'color': 'green'}, 'Passed SWIR1/B', False, opacity=0.7)
Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')
Map

Map(center=[-10.704552846449875, 25.788532369849722], controls=(WidgetControl(options=['position', 'transparen…

In [35]:
region3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]])

Map = geemap.Map()
Map.centerObject(region3, 11)

s = create_segments(region3, 0.5)

segments = ee.FeatureCollection(s).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
passed_ndmi_loss = filter_by_ndmi_loss(passed_sar_vh, 0.05)
passed_nir_swir2_loss = filter_by_nir_swir2_loss(passed_ndmi_loss, 0)
passed_swir1_b = filter_by_swir1_b(passed_nir_swir2_loss, 0.65)
passed_nir_g = filter_by_nir_g(passed_swir1_b, 0.45)

#composites = ee.ImageCollection(segments.map(create_median_composite))
composites = create_whole_composite(region3)

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', False, opacity=0.7)
# Map.addLayer(passed_nir_swir2_loss, {'color': 'yellow'}, 'Passed NIR/SWIR2 Loss', False, opacity=0.7)
# Map.addLayer(passed_swir1_b, {'color': 'green'}, 'Passed SWIR1/B', False, opacity=0.7)
Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')
Map

Map(center=[-10.62500765669145, 26.12500000000105], controls=(WidgetControl(options=['position', 'transparent_…